In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import re
import stanza
from collections import Counter
stanza.download('en')  

2020-04-29 00:17:40 INFO: Downloading default packages for language: en (English)...
2020-04-29 00:17:41 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-04-29 00:17:44 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [3]:
import itertools
import datasets
import pickle
from transformers import AutoModelWithLMHead, AutoTokenizer

In [4]:

def print_words(words, f):
    for word in words:
        word_str = [word.word]
        if word.pos:
            word_str.append(f"/{word.pos}/")
        if word.topic:
            word_str.append(f"[{word.topic}]")
        print(" ".join(word_str), file=f)
        print(f"\t{word.definition}", file=f)
        print(f"\t\"{word.example}\"{' |e|' if word.from_example_expansion else ''}", file=f)

        print("", file=f)

2020-04-28 23:37:24 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-04-28 23:37:24 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-04-28 23:37:24 INFO: Use device: gpu
2020-04-28 23:37:24 INFO: Loading: tokenize
2020-04-28 23:37:24 INFO: Loading: pos
2020-04-28 23:37:25 INFO: Done loading processors!


Counter({'VBD': 3, 'NNP': 0, 'VBN': 0, 'IN': 0, 'POS': 0, 'NN': 0, '.': 0, 'PRP': 0, 'CD': 0, 'CC': 0, 'VBZ': 0})


'VBD'

In [5]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = set((x.lower() for x in itertools.chain.from_iterable(
    [e.word] + e.derivatives
    for e in pickle.load(open(f"../data/en_dictionary_parsed_randomized.pickle", "rb")))
))
model = AutoModelWithLMHead.from_pretrained("../models/en_dictionary_parsed_lr_00001/checkpoint-120000").to("cuda:0")

2020-04-29 00:17:58 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-04-29 00:17:58 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-04-29 00:17:58 INFO: Use device: gpu
2020-04-29 00:17:58 INFO: Loading: tokenize
2020-04-29 00:17:59 INFO: Loading: pos
2020-04-29 00:18:00 INFO: Done loading processors!


In [15]:
prefix

'<|bod|>dasfdagsmn24134fd.s~!'

In [39]:
words, stats = datasets.ParsedDictionaryDefinitionDataset.generate_words(
    tokenizer, model,
    num=500,
    max_iterations=50, 
    blacklist=blacklist, 
    do_example_expansion=True, 
    generation_args=dict(
        top_k=300,
        num_return_sequences=50,
        max_length=256,
        do_sample=True,
    ),
    expansion_generation_overrides=dict(
        top_k=50,
        num_return_sequences=20,
        do_sample=True,
    ),
    num_expansion_candidates=20,
    example_match_pos_pipeline=nlp,
)

print(stats)
print()
print_words(words, sys.stdout)

2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-scholar '': NOUN
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-phobic '': NOUN
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-torrente '': PROPN
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-thriller '': ADJ
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' the solfato-type activity '': ADJ
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-type '': NOUN
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-femini in deltoid '': NOUN
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-anxious '': ADJ
2020-04-29 00:42:59 WARNING: No UPOS mapping for combining form -  solfato-  in ' solfato-mute '': NOUN
2020-04-29 00:42:59 WARN

iterations=50 | items_considered 1.00@2500, failed_match 0.00@11, blacklist_filtered 0.73@1825, seen_filtered 0.00@1, proper_noun_filtered 0.05@135, example_filtered 0.00@0, example_expansions 0.19@478, example_expansion_success 0.16@407, returned 0.18@457

sunkard /noun/
	the unhygienic drink of the Sunkard.
	"the Sunkard was made from unsweetened black liquor" |e|

re-reject /verb [with object]/
	put (conflict, disagreement, rumors, etc.) back into place
	"a review was re-rejected by the Institute of International Affairs" |e|

courier couriership /noun/
	a US carrier based in New York City.
	"Courier couriership services have grown dramatically" |e|

therium pyrophosphate /noun/
	the red phosphorylation of potassium hydroxide, responsible for this chemical alteration. It is formed by the breakdown of potassium uric acid into uric carbon dioxide.
	"therium pyrophosphate is required for the formation of ova" |e|

joomla /noun/
	a frothy sea stench.
	"the cold joomla may be the drink o